<a href="https://colab.research.google.com/github/mykon23/AI-2020/blob/master/Activities/Activity6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
!wget -O Baseball.csv https://github.com/mykon23/AI-2020/blob/master/Data/Baseball.csv?raw=True

--2020-09-12 18:24:17--  https://github.com/mykon23/AI-2020/blob/master/Data/Baseball.csv?raw=True
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/mykon23/AI-2020/raw/master/Data/Baseball.csv [following]
--2020-09-12 18:24:18--  https://github.com/mykon23/AI-2020/raw/master/Data/Baseball.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mykon23/AI-2020/master/Data/Baseball.csv [following]
--2020-09-12 18:24:18--  https://raw.githubusercontent.com/mykon23/AI-2020/master/Data/Baseball.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 20

In [77]:
import pandas as pd
data = pd.read_csv('Baseball.csv', delimiter=',')
data.head()

,Team,League,Year,RS,RA,W,OBP,SLG,BA,Playoffs,RankSeason,RankPlayoffs,G,OOBP,OSLG
0,ARI,NL,2012,734,688,81,0.328,0.418,0.259,0,NaN,NaN,162,0.317,0.415
1,ATL,NL,2012,700,600,94,0.320,0.389,0.247,1,4.0,5.0,162,0.306,0.378
2,BAL,AL,2012,712,705,93,0.311,0.417,0.247,1,5.0,4.0,162,0.315,0.403
3,BOS,AL,2012,734,806,69,0.315,0.415,0.260,0,NaN,NaN,162,0.331,0.428
4,CHC,NL,2012,613,759,61,0.302,0.378,0.240,0,NaN,NaN,162,0.335,0.424


In [78]:
##Get the features of interest
x_features = [ 'RS', 'RA', 'W', 'OBP', 'SLG', 'BA' ]
y_output = [ 'Playoffs' ]

##Only get the data from teams after 1994 when the wildcard was introduced
##Prior to the wildcard game, only four or two teams made it to playoffs instead of 8
work_data = data[ data['Year'] > 1994 ]
work_data = work_data[ x_features + y_output ]
print(work_data)

      RS   RA   W    OBP    SLG     BA  Playoffs
0    734  688  81  0.328  0.418  0.259         0
1    700  600  94  0.320  0.389  0.247         1
2    712  705  93  0.311  0.417  0.247         1
3    734  806  69  0.315  0.415  0.260         0
4    613  759  61  0.302  0.378  0.240         0
..   ...  ...  ..    ...    ...    ...       ...
501  993  895  85  0.366  0.484  0.287         0
502  752  862  68  0.331  0.388  0.253         0
503  759  706  88  0.330  0.407  0.267         1
504  928  799  90  0.358  0.469  0.284         1
505  766  809  74  0.331  0.420  0.259         0

[506 rows x 7 columns]


In [79]:
##Convert the working data into numpy array
import numpy as np
work_data = work_data.to_numpy().astype(np.float64)

In [80]:
X = work_data[:, :-1]
Y = work_data[:, -1]

In [81]:
##Standardization
mean = X.mean(axis = 0)
X -= mean
std = X.std(axis = 0)
X /= std

In [82]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(1, input_dim = len(X[0, :]), activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'rmsprop', metrics=['accuracy'])
model.fit(X, Y, epochs = 1024, verbose = 0)

In [83]:
##Get the preditions for the most current year in the data for 30 teams
prediction = model.predict(X)
for i in range(30):
  print(f'Validation: {Y[i]}, Prediction: {prediction[i].T[0]}')

Validation: 0.0, Prediction: 0.022229939699172974
Validation: 1.0, Prediction: 0.8881652355194092
Validation: 1.0, Prediction: 0.7530145645141602
Validation: 0.0, Prediction: 5.45075636182446e-05
Validation: 0.0, Prediction: 1.814902930163953e-06
Validation: 0.0, Prediction: 0.08313241600990295
Validation: 1.0, Prediction: 0.9590075016021729
Validation: 0.0, Prediction: 7.70863116485998e-05
Validation: 0.0, Prediction: 7.676392669964116e-06
Validation: 1.0, Prediction: 0.3772439956665039
Validation: 0.0, Prediction: 1.442006833940468e-07
Validation: 0.0, Prediction: 0.0002446174621582031
Validation: 0.0, Prediction: 0.3911699056625366
Validation: 0.0, Prediction: 0.169251948595047
Validation: 0.0, Prediction: 8.029433956835419e-05
Validation: 0.0, Prediction: 0.040606945753097534
Validation: 0.0, Prediction: 2.4847329768817872e-05
Validation: 0.0, Prediction: 0.0008864402770996094
Validation: 1.0, Prediction: 0.9291223883628845
Validation: 1.0, Prediction: 0.8526228666305542
Validation

The predicted values are compared to the validation values to acquire correctness.  Values that are greater than or equal to 0.5 shall map to 1.0 whereas values below 0.5 shall map to 0.0.  The values for logistic regression are reasonable as correct predictions are acquired for 27 of the 30 teams.  The incorrect predictions may be a result of the problem as the MLB favors division winners over teams with better records; i.e. The division winner may have a worse record than other second place teams, but will move onto the playoffs due to being the 'best' within their division. 